# **NLP2_Run3**

In [ ]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.6 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [ ]:

import openai
import os
from IPython.display import Markdown


### **Accessing the GPT4 API**

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']
openai.api_key = my_api_key


### **Generate Mathematical Model**

In [ ]:
problem = """A buyer needs to acquire 239,600,480 units of a product and is considering bids from five suppliers, labeled A through E, each of whom can only supply a portion of the total required amount.
Each vendor has proposed different pricing structures, incorporating both setup fees and variable unit costs that change based on the quantity ordered.

The buyer's objective is to allocate the order among these suppliers to minimize overall costs, accounting for both setup and unit costs.

Vendor A offers a set up cost of $3855.34 and a unit cost of $61.150 per thousand of units.
Vendor A can supply up to 33 million units.

Vendor B offers a set up cost of $125,804.84 if purchasing between 22,000,000-70,000,000 units from vendor B with a unit cost of $68.099 per thousand units.
If purchasing between 70,000,001-100,000,000 units from vendor B, the set up cost increases to $269304.84 and the unit cost sinks to $66.049 per thousand units.
If purchasing between 100,000,001-150,000,000 units from vendor B, the unit cost per thousand units further decreases to $64.099, but the set up cost increases to $464304.84.
If purchasing between 150,000,001 and 160,000,000 units from vendor B, the unit cost is $62.119 per thousand units and the set up cost equals $761304.84.

Vendor C offers set up costs of $13,456.00 and a unit cost of $62.019 per thousand units.
Vendor C can supply up to 165.6 million units. Vendor D offers set up costs of $6,583.98 and a unit cost of $72.488 for a set of thousand units.

Vendor D can supply up to 12 million units at a price of $72.488 per thousand units and with a set up cost of $6583.98.

Vendor E offers free set up if purchasing between 0 and 42 million units of vendor E with a unit price of $70.150 per thousand units.
If purchasing between 42,000,001 and 77 million units from vendor E, the unit cost starts at $68.150 per thousand units, but with every one million units purchased the price decreases at a rate of 0.05 percent. An additional set up cost of $84000 will be charged as well.

Note that zero units may be purchased from vendor B: otherwise no positive number of units less than 22,000,000 may be purchased."""

In [ ]:

client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the variables for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 2
)


In [ ]:
print(response1.choices[0].message.content)

Let's denote the total units purchased from each vendor as:

A = Units purchased from Vendor A.
B = Units purchased from Vendor B.
C = Units purchased from Vendor C.
D = Units purchased from Vendor D.
E = Units purchased from Vendor E.

We also need to define some binary variables to handle the setup costs for each vendor and the different pricing conditions:

a = 1 if any units are purchased from Vendor A, 0 otherwise.
b1 = 1 if between 22,000,000 and 70,000,000 units are purchased from Vendor B, 0 otherwise.
b2 = 1 if between 70,000,001 and 100,000,000 units are purchased from Vendor B, 0 otherwise.
b3 = 1 if between 100,000,001 and 150,000,000 units are purchased from Vendor B, 0 otherwise.
b4 = 1 if between 150,000,001 and 160,000,000 units are purchased from Vendor B, 0 otherwise.
c = 1 if any units are purchased from Vendor C, 0 otherwise.
d = 1 if any units are purchased from Vendor D, 0 otherwise.
e1 = 1 if between 0 and 42,000,000 units are purchased from Vendor E, 0 otherwise

In [ ]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the objective function for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [ ]:
print(response2.choices[0].message.content)

The objective function to minimize total cost will be as follows. Let's denote 10^3 by K,

Minimize Z = a*(3855.34 + 61.150*A/K) + b1*(125804.84 + 68.099*B/K) + b2*(269304.84 + 66.049*B/K) 
+ b3*(464304.84 + 64.099*B/K) + b4*(761304.84 + 62.119*B/K)
+ c*(13456 + 62.019*C/K) + d*(6583.98 + 72.488*D/K) 
+ e1*(70.150*E/K) + e2*(84000 + Ue2*E/K) - e2*E_discount*0.0005*Ue2*K 
where, the unit costs Ub1, Ub2, Ub3, Ub4, Ue1, Ue2 are constants representing the average unit costs of Vendor B and E's different tiers, and E_discount is a variable representing the total discount for purchasing from Vendor E.


In [ ]:
response3 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the constraints for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content +  response2.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [ ]:
print(response3.choices[0].message.content)

1) 0 <= A <= 33,000,000

2) B = 0 or 22,000,000 <= B <= 160,000,000

3) 0 <= C <= 165,600,000

4) 0 <= D <= 12,000,000

5) 0 <= E <= 77,000,000

6) A+B+C+D+E = 239,600,480 

7) a = 1 if A>0; a = 0 otherwise

8) b1 = 1 if 22,000,000 <= B <= 70,000,000; b1 = 0 otherwise

9) b2 = 1 if 70,000,001 <= B <= 100,000,000; b2 = 0 otherwise

10) b3 = 1 if 100,000,001 <= B <= 150,000,000; b3 = 0 otherwise

11) b4 = 1 if 150,000,001 <= B <= 160,000,000; b4 = 0 otherwise

12) c = 1 if C>0; c = 0 otherwise

13) d = 1 if D>0; d = 0 otherwise

14) e1 = 1 if 0 < E <= 42,000,000; e1 = 0 otherwise

15) e2 = 1 if 42,000,001 <= E <= 77,000,000; e2 = 0 otherwise 

16) E_discount = (E - 42,000,000) / 1,000,000  if E > 42,000,000; E_discount = 0 otherwise 



### **Generate Pyomo Code**

In [ ]:
response4 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content + response2.choices[0].message.content + response3.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [ ]:
Markdown(response4.choices[0].message.content)

Here is a possible pyomo implementation of your problem. Sample data used are the unit costs for different vendor tiers.

```python
from pyomo.environ import *

# create a model
model = ConcreteModel()

# define variables
model.A = Var(within=NonNegativeReals, bounds=(0, 33e6))
model.B = Var(within=NonNegativeReals, bounds=(22e6, 160e6))
model.C = Var(within=NonNegativeReals, bounds=(0, 165.6e6))
model.D = Var(within=NonNegativeReals, bounds=(0, 12e6))
model.E = Var(within=NonNegativeReals, bounds=(0, 77e6))

model.a = Var(within=Binary)
model.b1 = Var(within=Binary)
model.b2 = Var(within=Binary)
model.b3 = Var(within=Binary)
model.b4 = Var(within=Binary)
model.c = Var(within=Binary)
model.d = Var(within=Binary)
model.e1 = Var(within=Binary)
model.e2 = Var(within=Binary)

model.E_discount = Var(within=NonNegativeReals)

# define objective
model.value = Objective(
    expr = model.a * (3855.34 + 61.150*model.A/1000) + model.b1 * (125804.84 + 68.099*model.B/1000) + 
    model.b2 * (269304.84 + 66.049*model.B/1000) + model.b3 * (464304.84 + 64.099*model.B/1000) + 
    model.b4 * (761304.84 + 62.119*model.B/1000) + model.c * (13456 + 62.019*model.C/1000) + 
    model.d * (6583.98 + 72.488*model.D/1000) + model.e1 * (70.150 * model.E/1000) + 
    model.e2 * (84000 + 70.150*model.E/1000) - model.e2*model.E_discount*0.0005*70.150*1000, sense=minimize)

# define constraints
model.totalUnits = Constraint(expr = model.A + model.B + model.C + model.D + model.E == 239.6e6)

model.a1 = Constraint(expr = model.A <= 33e6*model.a)
model.b1b4 = Constraint(expr = model.b1 + model.b2 + model.b3 + model.b4 == model.B/22e6)
model.c1 = Constraint(expr = model.C <= 165.6e6*model.c)
model.d1 = Constraint(expr = model.D <= 12e6*model.d)
model.e1e2 = Constraint(expr = model.e1 + model.e2 == model.E/42e6)
model.discount = Constraint(expr = model.E_discount == (model.E - 42e6)/1e6)

# solve
SolverFactory('glpk').solve(model)

print("Value =", model.value())
print("A =", model.A())
print("B =", model.B())
print("C =", model.C())
print("D =", model.D())
print("E =", model.E())
```

Note: The above implementation is a simplified version of the problem. In reality, more care should be taken to properly couple the binary decision variables with the volume purchasing variables. The chosen bounds for the variables are approximations or simplifications as well.

### **Run the code Generated by GPT4**

In [ ]:
# Install Python API for AMPL
!pip install amplpy --upgrade

from amplpy import AMPL,ampl_notebook

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 35.1 MB/s eta 0:00:00


In [ ]:
_ = load_dotenv(find_dotenv('second_knitro_license.env'))
my_knitro_license = os.environ['knitro_license']

ampl = ampl_notebook(
    modules=["coin"],  # modules to install
    license_uuid=my_knitro_license,  # license to use
)

Licensed to AMPL Community Edition License for <cevikmelis14@gmail.com>.


In [ ]:
from pyomo.environ import *

# create a model
model = ConcreteModel()

# define variables
model.A = Var(within=NonNegativeReals, bounds=(0, 33e6))
model.B = Var(within=NonNegativeReals, bounds=(22e6, 160e6))
model.C = Var(within=NonNegativeReals, bounds=(0, 165.6e6))
model.D = Var(within=NonNegativeReals, bounds=(0, 12e6))
model.E = Var(within=NonNegativeReals, bounds=(0, 77e6))

model.a = Var(within=Binary)
model.b1 = Var(within=Binary)
model.b2 = Var(within=Binary)
model.b3 = Var(within=Binary)
model.b4 = Var(within=Binary)
model.c = Var(within=Binary)
model.d = Var(within=Binary)
model.e1 = Var(within=Binary)
model.e2 = Var(within=Binary)

model.E_discount = Var(within=NonNegativeReals)

# define objective
model.value = Objective(
    expr = model.a * (3855.34 + 61.150*model.A/1000) + model.b1 * (125804.84 + 68.099*model.B/1000) +
    model.b2 * (269304.84 + 66.049*model.B/1000) + model.b3 * (464304.84 + 64.099*model.B/1000) +
    model.b4 * (761304.84 + 62.119*model.B/1000) + model.c * (13456 + 62.019*model.C/1000) +
    model.d * (6583.98 + 72.488*model.D/1000) + model.e1 * (70.150 * model.E/1000) +
    model.e2 * (84000 + 70.150*model.E/1000) - model.e2*model.E_discount*0.0005*70.150*1000, sense=minimize)

# define constraints
model.totalUnits = Constraint(expr = model.A + model.B + model.C + model.D + model.E == 239.6e6)
model.b1b4 = Constraint(expr = model.b1 + model.b2 + model.b3 + model.b4 == model.B/22e6)
model.a1 = Constraint(expr = model.A <= 33e6*model.a)

model.c1 = Constraint(expr = model.C <= 165.6e6*model.c)
model.d1 = Constraint(expr = model.D <= 12e6*model.d)
model.e1e2 = Constraint(expr = model.e1 + model.e2 == model.E/42e6)
model.discount = Constraint(expr = model.E_discount == (model.E - 42e6)/1e6)

# solve
SolverFactory('couenne').solve(model)

print("Value =", model.value())
print("A =", model.A())
print("B =", model.B())
print("C =", model.C())
print("D =", model.D())
print("E =", model.E())

Value = 162845.3399999991
A = 2599999.9999999856
B = 160000000.0
C = 0.0
D = 0.0
E = 77000000.0


### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**

In [ ]:
from pyomo.environ import *

# create a model
model = ConcreteModel()

# define variables
model.A = Var(within=NonNegativeReals, bounds=(0, 33e6))
model.B = Var(within=NonNegativeReals, bounds=(22e6, 160e6))
model.C = Var(within=NonNegativeReals, bounds=(0, 165.6e6))
model.D = Var(within=NonNegativeReals, bounds=(0, 12e6))
model.E = Var(within=NonNegativeReals, bounds=(0, 77e6))

model.a = Var(within=Binary)
model.b1 = Var(within=Binary)
model.b2 = Var(within=Binary)
model.b3 = Var(within=Binary)
model.b4 = Var(within=Binary)
model.c = Var(within=Binary)
model.d = Var(within=Binary)
model.e1 = Var(within=Binary)
model.e2 = Var(within=Binary)

model.E_discount = Var(within=NonNegativeReals)
M=100000000000
# define objective
model.value = Objective(
    expr = model.a * (3855.34 + 61.150*model.A/1000) + model.b1 * (125804.84 + 68.099*model.B/1000) +
    model.b2 * (269304.84 + 66.049*model.B/1000) + model.b3 * (464304.84 + 64.099*model.B/1000) +
    model.b4 * (761304.84 + 62.119*model.B/1000) + model.c * (13456 + 62.019*model.C/1000) +
    model.d * (6583.98 + 72.488*model.D/1000) + model.e1 * (70.150 * model.E/1000) +
    model.e2 * (84000 + 70.150*model.E/1000) - model.e2*model.E_discount*0.0005*70.150*1000, sense=minimize)

# define constraints
model.totalUnits = Constraint(expr = model.A + model.B + model.C + model.D + model.E == 239600480)

model.a1 = Constraint(expr = model.A <= 33e6*model.a)

model.C1 = Constraint(expr=model.B >= 22_000_000 - M * (1 - model.b1))
model.C2 = Constraint(expr=model.B <= 70_000_000 + M * (1 - model.b1))
model.C3 = Constraint(expr=model.B >= 70_000_001 - M * (1 - model.b2))
model.C4 = Constraint(expr=model.B <= 100_000_000 + M * (1 - model.b2))
model.C5 = Constraint(expr=model.B >= 100_000_001 - M * (1 - model.b3))
model.C6 = Constraint(expr=model.B <= 150_000_000 + M * (1 - model.b3))
model.C7 = Constraint(expr=model.B >= 150_000_001 - M * (1 - model.b4))
model.C8 = Constraint(expr=model.B <= 160_000_000 + M * (1 - model.b4))

model.c1 = Constraint(expr = model.C <= 165.6e6*model.c)
model.d1 = Constraint(expr = model.D <= 12e6*model.d)

model.e1_constraint2 = Constraint(expr=model.E <= 42_000_000 + M * (1 - model.e1))

model.e2_constraint1 = Constraint(expr=model.E >= 42_000_001 - M * (1 - model.e2))
model.e2_constraint2 = Constraint(expr=model.E <= 77_000_000 + M * (1 - model.e2))

model.discount = Constraint(expr = model.E_discount == (model.E - 42e6)/1e6)

# solve
SolverFactory('couenne').solve(model)

print("Value =", model.value())
print("A =", model.A())
print("B =", model.B())
print("C =", model.C())
print("D =", model.D())
print("E =", model.E())

Value = 162874.69200001014
A = 2600480.0000001662
B = 159999999.99999982
C = 0.0
D = 0.0
E = 77000000.0
